In [24]:
import fitz 
from pathlib import Path
from PIL import Image
import pytesseract
import os
import shutil
from PIL import Image, ImageEnhance, ImageFilter
import os
import re
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions
from pymongo import MongoClient
import uuid


In [6]:

# Caminho dos PDFs
pdf_folder_path = Path("../pdf_documents")
pdf_files = [f for f in os.listdir(pdf_folder_path) if f.endswith(".pdf")]

# Caminho de saída para as imagens
output_dir = Path("../extracted_images")
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Extraindo imagens para: {output_dir.resolve()}\n")

# Função para extrair imagens embutidas de um PDF
def extract_images_from_pdf(pdf_path: Path):
    pdf_name = pdf_path.stem
    with fitz.open(pdf_path) as doc:
        for page_number in range(len(doc)):
            page = doc[page_number]
            images = page.get_images(full=True)
            
            for img_index, img in enumerate(images):
                xref = img[0]
                try:
                    base_image = doc.extract_image(xref)
                    image_bytes = base_image["image"]
                    image_ext = base_image["ext"]

                    # Nome do arquivo: nomeDoArquivo_paginaX_imgY.ext
                    image_filename = f"{pdf_name}_pagina{page_number+1}_img{img_index+1}.{image_ext}"
                    image_path = output_dir / image_filename

                    with open(image_path, "wb") as f:
                        f.write(image_bytes)
                    
                    print(f"Imagem salva: {image_filename}")
                except Exception as e:
                    print(f"Erro ao extrair imagem {img_index+1} da página {page_number+1} em {pdf_name}: {e}")

# Processar todos os PDFs
for pdf_file in pdf_files:
    pdf_path = pdf_folder_path / pdf_file
    print(f"\nProcessando: {pdf_file}")
    extract_images_from_pdf(pdf_path)

print("\nExtração de imagens concluída.")


Extraindo imagens para: /home/stoiccode/gitRepositories/Database-Chroma-RAG-Project/extracted_images


📄 Processando: RIMA_example1.pdf
Imagem salva: RIMA_example1_pagina1_img1.jpeg
Imagem salva: RIMA_example1_pagina2_img1.jpeg
Imagem salva: RIMA_example1_pagina3_img1.jpeg
Imagem salva: RIMA_example1_pagina3_img2.jpeg
Imagem salva: RIMA_example1_pagina3_img3.jpeg
Imagem salva: RIMA_example1_pagina3_img4.jpeg
Imagem salva: RIMA_example1_pagina3_img5.jpeg
Imagem salva: RIMA_example1_pagina4_img1.jpeg
Imagem salva: RIMA_example1_pagina5_img1.jpeg
Imagem salva: RIMA_example1_pagina5_img2.jpeg
Imagem salva: RIMA_example1_pagina8_img1.jpeg
Imagem salva: RIMA_example1_pagina8_img2.jpeg
Imagem salva: RIMA_example1_pagina9_img1.jpeg
Imagem salva: RIMA_example1_pagina9_img2.jpeg
Imagem salva: RIMA_example1_pagina10_img1.jpeg
Imagem salva: RIMA_example1_pagina10_img2.jpeg
Imagem salva: RIMA_example1_pagina10_img3.jpeg
Imagem salva: RIMA_example1_pagina11_img1.jpeg
Imagem salva: RIMA_example1_pagi

In [7]:

# Palavras-chave comuns em gráficos/mapas
KEYWORDS = ['gráfico', 'grafico', 'mapa', 'legenda', 'fonte', 'dados', 'eixo', 'percentual', 'escala', 'curva']

# Pastas
input_folder = Path("../extracted_images")
output_folder = Path("../filtered_images")
output_folder.mkdir(exist_ok=True)

# Função para verificar se imagem contém palavras-chave
def contains_keywords(image_path):
    try:
        text = pytesseract.image_to_string(Image.open(image_path), lang='por')
        return any(keyword in text.lower() for keyword in KEYWORDS)
    except Exception as e:
        print(f"[ERRO] {image_path.name}: {e}")
        return False

# Processamento
total = 0
movidas = 0

for image_path in input_folder.glob("*"):
    if image_path.is_dir():
        continue
    total += 1
    if contains_keywords(image_path):
        shutil.copy(image_path, output_folder / image_path.name)
        movidas += 1
        print(f"[OK] {image_path.name} => filtrada e copiada.")

print(f"\nTotal de imagens processadas: {total}")
print(f"Total de imagens copiadas: {movidas}")


[OK] EIA_example1_pagina35_img1.jpeg => filtrada e copiada.
[OK] EIA_example1_pagina60_img2.jpeg => filtrada e copiada.
[OK] EIA_example1_pagina60_img3.jpeg => filtrada e copiada.
[OK] EIA_example1_pagina61_img2.jpeg => filtrada e copiada.
[OK] EIA_example1_pagina63_img2.jpeg => filtrada e copiada.
[OK] EIA_example1_pagina96_img2.jpeg => filtrada e copiada.
[OK] EIA_example1_pagina98_img2.jpeg => filtrada e copiada.
[OK] EIA_example1_pagina100_img2.jpeg => filtrada e copiada.
[OK] EIA_example1_pagina108_img2.jpeg => filtrada e copiada.
[OK] EIA_example1_pagina119_img3.jpeg => filtrada e copiada.
[OK] EIA_example1_pagina121_img2.jpeg => filtrada e copiada.
[OK] EIA_example1_pagina122_img2.jpeg => filtrada e copiada.
[OK] EIA_example1_pagina130_img2.jpeg => filtrada e copiada.
[OK] EIA_example1_pagina136_img2.jpeg => filtrada e copiada.
[OK] EIA_example1_pagina137_img2.jpeg => filtrada e copiada.
[OK] RIMA_example1_pagina4_img1.jpeg => filtrada e copiada.
[OK] RIMA_example1_pagina8_img1.

In [8]:
input_dir = Path("../filtered_images")
output_dir = Path("../described_images")
output_dir.mkdir(parents=True, exist_ok=True)

for image_path in input_dir.glob("*.[pjPJ]*[npNP]*[gG]*"):  # PNG ou JPG
    try:
        image = Image.open(image_path).convert("L")  # escala de cinza
        image = image.filter(ImageFilter.SHARPEN)
        image = ImageEnhance.Contrast(image).enhance(2)

        text = pytesseract.image_to_string(image, lang="por")

        # Salva o texto extraído
        output_txt_path = output_dir / (image_path.stem + ".txt")
        with open(output_txt_path, "w", encoding="utf-8") as f:
            f.write(text)

        print(f"[OK] {image_path.name}")

    except Exception as e:
        print(f"[ERRO] {image_path.name}: {e}")


[OK] EIA_example1_pagina35_img1.jpeg
[OK] EIA_example1_pagina60_img2.jpeg
[OK] EIA_example1_pagina60_img3.jpeg
[OK] EIA_example1_pagina61_img2.jpeg
[OK] EIA_example1_pagina63_img2.jpeg
[OK] EIA_example1_pagina96_img2.jpeg
[OK] EIA_example1_pagina98_img2.jpeg
[OK] EIA_example1_pagina100_img2.jpeg
[OK] EIA_example1_pagina108_img2.jpeg
[OK] EIA_example1_pagina119_img3.jpeg
[OK] EIA_example1_pagina121_img2.jpeg
[OK] EIA_example1_pagina122_img2.jpeg
[OK] EIA_example1_pagina136_img2.jpeg
[OK] EIA_example1_pagina137_img2.jpeg
[OK] RIMA_example1_pagina4_img1.jpeg
[OK] RIMA_example1_pagina8_img1.jpeg
[OK] RIMA_example1_pagina18_img3.jpeg
[OK] RIMA_example1_pagina19_img1.jpeg
[OK] EIA_example1_pagina130_img2.jpeg


In [26]:
known_places = [
    # Estados
    "Acre", "Alagoas", "Amapá", "Amazonas", "Bahia", "Ceará", "Distrito Federal", "Espírito Santo", "Goiás",
    "Maranhão", "Mato Grosso", "Mato Grosso do Sul", "Minas Gerais", "Pará", "Paraíba", "Paraná", "Pernambuco",
    "Piauí", "Rio de Janeiro", "Rio Grande do Norte", "Rio Grande do Sul", "Rondônia", "Roraima", "Santa Catarina",
    "São Paulo", "Sergipe", "Tocantins",

    # Principais cidades
    "São Paulo", "Rio de Janeiro", "Belo Horizonte", "Brasília", "Salvador", "Curitiba", "Fortaleza", "Manaus",
    "Recife", "Campinas", "Goiânia", "Belém", "Porto Alegre", "São Luís", "Natal", "Maceió", "João Pessoa",
    "Aracaju", "Cuiabá", "Campo Grande", "Florianópolis", "Vitória", "Macapá", "Rio Branco", "Palmas"
]

In [25]:
def extract_location_metadata(text: str) -> dict:
    metadata = {}

    # Coordenadas UTM simples
    utm_matches = re.findall(r"\b\d{6,7}\b", text)
    if utm_matches:
        metadata["utm_coords"] = ", ".join(set(utm_matches))

    places_found = [p for p in known_places if p.lower() in text.lower()]
    if places_found:
        metadata["places"] = ", ".join(sorted(set(places_found)))

    return metadata


In [11]:

# Inicializa o modelo de embeddings local
model = SentenceTransformer("all-MiniLM-L6-v2")

# Caminho para as descrições
desc_dir = Path("../described_images")
desc_files = list(desc_dir.glob("*.txt"))

# Chroma DB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection("image_descriptions")

# MongoDB
mongo_client = MongoClient("mongodb://localhost:27017/")
mongo_db = mongo_client["rag_db"]
image_chunks = mongo_db["image_chunks"]

# Processamento
for txt_file in desc_files:
    with open(txt_file, "r", encoding="utf-8") as f:
        description = f.read().strip()

    if not description:
        continue

    embedding = model.encode(description).tolist()
    doc_id = str(uuid.uuid4())

    location_metadata = extract_location_metadata(description)
    # Armazena no Chroma
    collection.add(
        documents=[description],
        embeddings=[embedding],
        ids=[doc_id],
        metadatas=[{
            "source": txt_file.name,
            **location_metadata
        }]
    )
    # Armazena no MongoDB
    image_chunks.insert_one({
        "id": doc_id,
        "source": txt_file.name,
        "text": description,
        "embedding": embedding
    })


    print(f"[OK] {txt_file.name} armazenado")




[OK] EIA_example1_pagina35_img1.txt armazenado
[OK] EIA_example1_pagina60_img2.txt armazenado
[OK] EIA_example1_pagina60_img3.txt armazenado
[OK] EIA_example1_pagina61_img2.txt armazenado
[OK] EIA_example1_pagina63_img2.txt armazenado
[OK] EIA_example1_pagina96_img2.txt armazenado
[OK] EIA_example1_pagina98_img2.txt armazenado
[OK] EIA_example1_pagina100_img2.txt armazenado
[OK] EIA_example1_pagina108_img2.txt armazenado
[OK] EIA_example1_pagina119_img3.txt armazenado
[OK] EIA_example1_pagina121_img2.txt armazenado
[OK] EIA_example1_pagina122_img2.txt armazenado
[OK] EIA_example1_pagina136_img2.txt armazenado
[OK] EIA_example1_pagina137_img2.txt armazenado
[OK] RIMA_example1_pagina4_img1.txt armazenado
[OK] RIMA_example1_pagina8_img1.txt armazenado
[OK] RIMA_example1_pagina18_img3.txt armazenado
[OK] RIMA_example1_pagina19_img1.txt armazenado
[OK] EIA_example1_pagina130_img2.txt armazenado


In [15]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [21]:
collections = chroma_client.list_collections()

# Exibe os nomes das collections
for c in collections:
    print(c.name)

rag_embeddings
image_contexts
image_descriptions
